In [68]:
#Fashion-MNIST，是去年8月底德国研究机构Zalando Research发布的一个数据集，其中训练集包含60000个样本，
#测试集包含10000个样本，分为10类。样本都来自日常穿着的衣裤鞋包，每一个都是28×28的灰度图。
#T恤 0 裤子 1 套头衫 2 裙子 3 外套 4 凉鞋 5 衬衫 6 运动鞋 7 包 8 踝靴 9

import tensorflow as tf
import numpy as np
from keras.layers import Dense,Dropout,Activation,Conv2D,MaxPooling2D,Flatten
from keras.optimizers import SGD
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from keras.layers.normalization import  BatchNormalization

In [69]:
(x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

x_train shape: (60000, 28, 28) y_train shape: (60000,)


In [70]:
#将数据进行归一化,并分为训练集和测试集
from keras.utils import to_categorical
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255
#(x_train,x_valid) = x_train[5000:],x_train[:5000]
#(y_train,y_valid) = y_train[5000:],y_train[:5000]
#reshape input data from (28,28) to (28,28,1)
w,h = 28,28
x_train = x_train.reshape(x_train.shape[0],w,h,1)
#x_valid = x_valid.reshape(x_valid.shape[0],w,h,1)
x_test = x_test.reshape(x_test.shape[0],w,h,1)

#one-hot编码
y_train = to_categorical(y_train,10)
#y_valid = to_categorical(y_valid,10)
y_test = to_categorical(y_test,10)

#print(y_train)
#print
print("x_train shape:", x_train.shape,"y_train shape:",y_train.shape)
print(x_train.shape[0],'train_set')
#print(x_valid.shape[0],'validation set')
print(x_test.shape[0],'test set')

[[ 0.  0.  0. ...,  0.  0.  1.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
x_train shape: (60000, 28, 28, 1) y_train shape: (60000, 10)
60000 train_set
10000 test set


In [75]:
#使用序贯模型
from keras.models import Sequential
model = Sequential()
model.add(Conv2D(filters = 64,kernel_size = 2,padding = 'same',activation = 'relu',input_shape = (28,28,1)))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 64,kernel_size = 2,padding = 'same',activation = 'relu'))
model.add(MaxPooling2D(pool_size = 2))
model.add(Dropout(0.2))

model.add(Flatten())#把多维的输入一维化
model.add(Dense(100,activation = 'tanh'))
model.add(BatchNormalization(momentum=0.9))
model.add(Dropout(0.2))

model.add(Dense(100,activation = 'tanh'))
model.add(BatchNormalization(momentum=0.9))
model.add(Dropout(0.2))

model.add(Dense(100,activation = 'tanh'))
model.add(BatchNormalization(momentum=0.9))
model.add(Dropout(0.2))

model.add(Dense(10,activation = 'softmax'))
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 28, 28, 64)        320       
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_47 (Dropout)         (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 14, 14, 64)        16448     
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_48 (Dropout)         (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 3136)              0         
__________

In [78]:
#编译模型
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose = 1, save_best_only=True)
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
#训练模型
model.fit(x_train,y_train,batch_size = 128,epochs = 10,verbose = 0,shuffle=True,validation_split = 0.33,callbacks=[checkpointer])



Epoch 00001: val_loss improved from inf to 0.37524, saving model to model.weights.best.hdf5

Epoch 00002: val_loss improved from 0.37524 to 0.32432, saving model to model.weights.best.hdf5

Epoch 00003: val_loss improved from 0.32432 to 0.30191, saving model to model.weights.best.hdf5

Epoch 00004: val_loss improved from 0.30191 to 0.27682, saving model to model.weights.best.hdf5

Epoch 00005: val_loss improved from 0.27682 to 0.27265, saving model to model.weights.best.hdf5

Epoch 00006: val_loss improved from 0.27265 to 0.25467, saving model to model.weights.best.hdf5

Epoch 00007: val_loss did not improve

Epoch 00008: val_loss did not improve

Epoch 00009: val_loss improved from 0.25467 to 0.24323, saving model to model.weights.best.hdf5

Epoch 00010: val_loss improved from 0.24323 to 0.23796, saving model to model.weights.best.hdf5


In [79]:
# Load the weights with the best validation accuracy
model.load_weights('model.weights.best.hdf5')

In [80]:
# Evaluate the model on test set
score = model.evaluate(x_test, y_test, verbose=0)

# Print test accuracy
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.9105
